In [5]:
import pandas as pd

input_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\combined_categorized_transactions.csv"  # Replace with your actual file name
output_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\updated_combined_transactions.csv"

# CSV file
df = pd.read_csv(input_file)

# 'Sub-category' update where 'Açıklama' contains "Gıda"
# at first the hierarchy I built was problematic that some dining outs got categorized as market
df.loc[df["Açıklama"].str.contains("Gıda", na=False), "Sub-category"] = "Dining out - food"

# updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"Rows updated where 'Açıklama' contains 'Gıda'. Updated file saved to '{output_file}'.")


Rows updated where 'Açıklama' contains 'Gıda'. Updated file saved to 'C:\Users\Lenovo\Desktop\DSA210_raw_Data\updated_combined_transactions.csv'.


In [ ]:
# the mistake : it should go forward, not backwarded

In [16]:
import pandas as pd

input_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\updated_combined_transactions.csv"
output_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\distributed_combined_transactions.csv"

df = pd.read_csv(input_file)

# required columns exist
required_columns = ["Tarih", "Tutar", "Açıklama", "Sub-category"]
if not set(required_columns).issubset(df.columns):
    raise ValueError(f"The input CSV must have the following columns: {required_columns}")

# 'Tarih' is converted to datetime and sort by date
df["Tarih"] = pd.to_datetime(df["Tarih"], format="%Y-%m-%d-%H.%M.%S.%f")
df = df.sort_values(by="Tarih").reset_index(drop=True)

# placeholders
adjusted_rows = []
cafeteria_chargeups = []
days_to_distribute = []
processed_dates = set()
processed_rows = set()

# the buffer for cafeteria food in case of a charge-up
cafeteria_buffer = 0
current_date = df.loc[0, "Tarih"].date()  # initialization with the first date

# iterate over all rows by date
for i in range(len(df)):
    row = df.iloc[i]
    row_date = row["Tarih"].date()
    
    # date change detect
    if row_date != current_date or i == (len(df) - 1):
        # process rows for the previous date
        for j in processed_rows:
            inner_row = df.iloc[j]
            inner_date = inner_row["Tarih"].date()
            
            # prioritize cafeteria food charge-up
            if inner_row["Sub-category"] == "Cafeteria food":
                # distribute the buffer (if available) the previous to the following days
                days_to_distribute.remove(current_date) if current_date in days_to_distribute else None
                if cafeteria_buffer > 0 and len(days_to_distribute) > 0:
                    distributed_amount = cafeteria_buffer / len(days_to_distribute)
                    for day in days_to_distribute:
                        adjusted_rows.append({
                            "Tarih": pd.Timestamp(day),
                            "Tutar": -distributed_amount,
                            "Açıklama": "Distributed Cafeteria food",
                            "Sub-category": "Cafeteria food"
                        })
                
                cafeteria_chargeups.append(j)
                cafeteria_buffer = abs(inner_row["Tutar"])
                days_to_distribute = [current_date]
                
                processed_dates.clear()
                processed_dates.add(current_date)
                break  # stop inner loop to prioritize charge-up
            
            # skip "Dining out - food" days
            elif inner_row["Sub-category"] == "Dining out - food":
                processed_dates.add(current_date)
                days_to_distribute.remove(current_date) if current_date in days_to_distribute else None
            
            # add day for future distribution
            elif current_date not in processed_dates:
                days_to_distribute.append(current_date)
                processed_dates.add(current_date)
        
        # end of the day (for rows)
        processed_rows.clear()
        current_date = row_date
    
    processed_rows.add(i)


# final buffer distribution
if cafeteria_buffer > 0 and len(days_to_distribute) > 0:
    distributed_amount = cafeteria_buffer / len(days_to_distribute)
    for day in days_to_distribute:
        adjusted_rows.append({
            "Tarih": pd.Timestamp(day),
            "Tutar": -distributed_amount,
            "Açıklama": "Distributed Cafeteria food",
            "Sub-category": "Cafeteria food"
        })

# append distributed rows to the original DataFrame
distributed_df = pd.DataFrame(adjusted_rows)

# update "Tutar" to 0 for charge-up rows
df.loc[cafeteria_chargeups, "Tutar"] = 0

final_df = pd.concat([df, distributed_df]).sort_values(by="Tarih").reset_index(drop=True)


final_df = final_df.sort_values(by="Tarih", ascending=False).reset_index(drop=True)
final_df.to_csv(output_file, index=False)

print(f"Distribution complete. Results saved to '{output_file}'.")


Distribution complete. Results saved to 'C:\Users\Lenovo\Desktop\DSA210_raw_Data\distributed_combined_transactions.csv'.


In [17]:
import pandas as pd

# sub-category mapping to generals
general_category_mapping = {
    "Cafeteria food": "Essential",
    "Market – grocery": "Essential",
    "Education": "Essential",
    "Savings": "Essential",
    "Entertainment": "Discretionary",
    "Dining out - food": "Discretionary",
    "Dining out - coffee": "Discretionary",
    "Fashion": "Discretionary",
    "Transportation": "Essential",
    "ATM Use": "Essential",
    "Grant": "Income",
    "Allowance": "Income",
    "Others": "Discretionary"
}

# function to determine the general category
def determine_general_category(sub_category):
    return general_category_mapping.get(sub_category, "Discretionary") 
    # if a sub-category not found, assign it right to "Discretionary"
    # dict.get(key, default_value)
    
input_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\distributed_combined_transactions.csv" 
output_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\generally_categorized_transactions.csv"

df = pd.read_csv(input_file)

# if the "Sub-category" exists
if "Sub-category" in df.columns:
    # new column "General Category" based on the sub-category
    df["General Category"] = df["Sub-category"].apply(determine_general_category)
else:
    raise ValueError("The 'Sub-category' column is missing from the input CSV file.")

df.to_csv(output_file, index=False)

print(f"Categorization complete. Results saved to {output_file}")


Categorization complete. Results saved to C:\Users\Lenovo\Desktop\DSA210_raw_Data\generally_categorized_transactions.csv


In [18]:
import pandas as pd

input_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\generally_categorized_transactions.csv"
output_file = r"C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_transactions.csv"

df = pd.read_csv(input_file)

# 'Tarih' is datetime
df["Tarih"] = pd.to_datetime(df["Tarih"], errors="coerce")

# if 'Açıklama' is 'Distributed Cafeteria food'
condition = df["Açıklama"] == "Distributed Cafeteria food"

# updates based on the year and 'Tutar' value
for index, row in df.loc[condition].iterrows():
    year = row["Tarih"].year
    tutar_abs = abs(row["Tutar"])
    
    # Check for year and value thresholds
    if year == 2022 and tutar_abs < 40:
        df.at[index, "Tutar"] = -50
    elif year == 2023 and tutar_abs < 50:
        df.at[index, "Tutar"] = -60
    elif year == 2024 and tutar_abs < 80:
        df.at[index, "Tutar"] = -100

df.to_csv(output_file, index=False)

print(f"Updates complete. Results saved to '{output_file}'.")


Updates complete. Results saved to 'C:\Users\Lenovo\Desktop\DSA210_raw_Data\final_categorized_transactions.csv'.
